- Categories of Models
  - Generative Model
    - Definition
      - A model generating new data samples similar to training data
         -> Learn models based on knowledge from training data to generate new data (predictions)
    - Types of Generative Model
      - Restricted Boltzmann Machines (RBM)
      - Variational Autoencoder (VAE)
      - Generative Adversarial Network (GAN)
      - Hidden Markov Model (HMM)
      
  - Deterministic Model
    - Definition
      - A model generating the exact same output when given input data is the same
        -> Resembling consistent outputs solely based on input values without random elements
    - Types of Deterministic Model
      - Deep Nueral Network (DNN)
      - Convolutional Neural Network (CNN)
      - Long Short-Term Memory (LSTM)

- Differences
  - Boltzmann Machine (BM)
    - A fully connected and undirected graphical model
      -> Visiable units and hidden units are interconnected
    
  - Restricted Boltzmann Machine (RBM)
    - A bipartite graph with visiable and hidden units
      -> No connection between units within the same layer

### Preparation

In [6]:
# Import modules
import torch
import torch.nn as nn

import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.utils as vutils


### Define a Restricted Boltzmann Machine (RBM) Class

In [2]:
# Define a RBM class
class RBM(nn.Module):
    
    # Initialize a class
    def __init__(self, visiable_size, hidden_size):
        # Initialize the class from superclass
        super(RBM, self).__init__()
        
        # Initialize weight matrix
        self.W = nn.Parameter(torch.randn(visible_size, hidden_size))
        
        # Initialize bias for visible and hidden layers
        self.v_bias = nn.Parameter(torch.randn(visible_size))
        self.h_bias = nn.Parameter(torch.randn(hidden_size))
        
    # Define a propagation
    def forward(self, x):
        # Calculate probability of hidden layer using Sigmoid function
        hidden_prob = torch.sigmoid(torch.matmul(x, self.W) + self.h_bias)
                                         # 'matmul()': Matrix Multiplication
        
        # Binary state of hidden layers from activatino probability using a Bernoulli distribution
        hidden_state = torch.bernoulli(hidden_prob)
        
        # Calculate activation probability of visible units
        visible_prob = torch.sigmoid(torch.matmul(hidden_state,
                                                  torch.transpose(self.W, 0, 1) + self.v_bias))
        return visible_prob, hidden_state
        

### Load Dataset: MNIST

In [8]:
# Set transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

# Download Dataset
train_dataset = torchvision.datasets.MNIST(root = './data/0630-MNIST',
                                           train = True,
                                           transform = transform,
                                           download = True)

# Load dataset
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = 64,
                                           shuffle = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/0630-MNIST\MNIST\raw\train-images-idx3-ubyte.gz to ./data/0630-MNIST\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/0630-MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to ./data/0630-MNIST\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/0630-MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/0630-MNIST\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/0630-MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/0630-MNIST\MNIST\raw



### Set Parameters

In [12]:
# Set parameters
visible_size = 784  # Image size of MNIST: 28x28=784
hidden_size = 256
lr = 0.05

# Set model
rbm = RBM(visible_size, hidden_size)

# Set Loss Function
criterion = nn.BCELoss()

# Set Optimizer
optimizer = torch.optim.SGD(rbm.parameters(), lr = lr)

### Fit Models

In [13]:
# Fit the RBM model
num_epochs = 10
for epoch in range(num_epochs):
    
    for images, _ in train_loader:
        # Set inputs
        inputs = images.view(-1, visible_size)

        # Forward
        visible_prob, _ = rbm(inputs)

        # Calculate loss
        loss = criterion(visible_prob, inputs)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Display logs
    print(f'Epoch [{epoch+1} / {num_epochs}], Loss: {loss.item():.4f}')
    
    # Save images of weights as files
    vutils.save_image(rbm.W.view(hidden_size, 1, 28, 28),
                      f'weights_epoch_{epoch+1}.png',
                      normalize = True)
    
    # Save images of inputs and outputs as files
    inputs_display = inputs.view(-1, 1, 28, 28)
    outputs_display = visible_prob.view(-1, 1, 28, 28)
    comparison = torch.cat([inputs_display, outputs_display],
                            dim = 3)
    vutils.save_image(comparison,
                      f'reconstruction_epoch_{epoch+1}.png',
                      normalize = True)

Epoch [1 / 10], Loss: 43.4761
Epoch [2 / 10], Loss: 41.9570
Epoch [3 / 10], Loss: 40.8416
Epoch [4 / 10], Loss: 40.2589
Epoch [5 / 10], Loss: 40.7201
Epoch [6 / 10], Loss: 40.2960
Epoch [7 / 10], Loss: 40.8885
Epoch [8 / 10], Loss: 40.1883
Epoch [9 / 10], Loss: 40.5112
Epoch [10 / 10], Loss: 40.9633
